In [16]:
import requests as req
from pprint import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import getpass
import time
import math
from pymongo import MongoClient

In [17]:
vacancy_data = []

In [31]:
def pars_sj():
    main_link = 'https://www.superjob.ru'
    u_a = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0',
           'Accept':'*/*'}
    vacancy = '/vacancy/search/?keywords=Таксист'
    page = req.get(main_link+vacancy, headers=u_a)
    html = bs(page.text, 'lxml')

    vac_block = html.findAll('div',{'class':'f-test-vacancy-item'})
    
    for vac in vac_block:
        vac_link = vac.find('a',{'class':'_3dPok'})['href']
        vac_page = bs(req.get(main_link+vac_link, headers=u_a).text, 'lxml')
        vac_name = vac_page.find('h1',{'class':'_3mfro'}).getText()
        salary = vac_page.find('span', {'class':'ZON4b'}).getText()
        
        if salary[0:2] == 'от':
            min_salary = vac_page.find('span', {'class':'ZON4b'}).findChildren()[0].getText()
            max_salary = None
            currency = vac_page.find('span', {'class':'ZON4b'}).findChildren()[1].getText()
        elif salary[0:2] == 'до':
            max_salary = vac_page.find('span', {'class':'ZON4b'}).findChildren()[0].getText()
            min_salary = None
            currency = vac_page.find('span', {'class':'ZON4b'}).findChildren()[1].getText()
        else:
            max_salary = None
            min_salary = None
            currency = None
        vacancy_data.append({'URL':main_link,
                     'VAC_URL':main_link+vac_link,
                     'Должность':vac_name,
                     'Минимальная зарплата':min_salary,
                     'Максимальная зарплата':max_salary,
                     'Валюта':currency})


In [32]:
def pars_hh ():
    main_link = 'https://rostov.hh.ru'
    vacancy = '/search/vacancy?clusters=true&area=76&enable_snippets=true&salary=&st=searchVacancy&text=Такси&from=suggest_post'
    u_a = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0',
           'Accept':'*/*'}
    session = req.Session()
    page = session.get(main_link+vacancy, headers=u_a)
    html = bs(page.text, 'lxml')
    
    page_count = math.ceil(int(html.find('h1', {'data-qa':'page-title'}).getText().split()[0])/20)
    
    for i in range(page_count):
    
        vac_block = html.find('div', {'class':'vacancy-serp'}).findAll('div', {'class':'vacancy-serp-item'})
    
        for vac in vac_block:    
            vac_link = vac.find('a', {'class':'HH-LinkModifier'})['href']
            vac_page = bs(req.get(vac_link, headers=u_a).text, 'lxml')
            vac_name = vac_page.find('h1', {'data-qa':'vacancy-title'}).getText()
            salary = vac_page.find('div', {'class':'vacancy-title'}).find('p', {'class':'vacancy-salary'}).getText().split()

            if salary[0] == 'от' and salary[3] != 'до':
                min_salary = salary[1]+salary[2]
                max_salary = None
                currency = '₽'

            elif salary[0] == 'от' and salary[3] == 'до':
                min_salary = salary[1]+salary[2]
                max_salary = salary[4]+salary[5]
                currency = '₽'

            else:
                min_salary = None
                max_salary = None
                currency = None

            vacancy_data.append({'URL':main_link,
                         'VAC_URL':vac_link,
                         'Должность':vac_name,
                         'Минимальная зарплата':min_salary,
                         'Максимальная зарплата':max_salary,
                         'Валюта':currency})

        vacancy = html.find('a',{'class':'HH-Pager-Controls-Next'})['href']



In [ ]:
pars_sj()
pars_hh()

In [ ]:
df = pd.DataFrame(vacancy_data) 

In [ ]:
client = MongoClient('localhost', 27017)
db = client['jobs']
collection = db.col_1

collection.insert_one({'URL':'main_link',
                         'VAC_URL':"vac_link",
                         'Должность':'vac_name',
                         'Минимальная зарплата':'min_salary',
                         'Максимальная зарплата':'max_salary',
                         'Валюта':'currency'})
for i in vacancy_data:
    
    if collection.find({'VAC_URL':i['VAC_URL']}):
        collection.update_one({'VAC_URL':i['VAC_URL']},{'$set':{'Минимальная зарплата':i['Минимальная зарплата']}})
    else:
        collection.insert_one(i)

In [21]:
find_salary = collection.find({'Минимальная зарплата':{'$gt':100000}})

In [22]:
find_salary